# Model Analysis

The script guides through the counting of reactions in each compartment and the generation of the Venn diagramm for reaction overlap of the two K. phaffii models iMT1026, iRY1243 and the O. polymorpha model iUL909.

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model
from cobra import Reaction

Today = date.today().strftime('%y%m%d')

In [2]:
def CoSub(Flux, C1, C2, R):
    F1 = Flux / (C1*(1+1/R))
    F2 = F1*C1 / (R*C2)
    return F1, F2

def countCinFormula(formula):
    return int(re.search(r'C(.*)H',formula).group(1)) if re.search(r'C(.*)H',formula).group(1) else int(1)

def set_ObjiUma(model, Product):
        model.add_boundary(model.metabolites.get_by_id(Product), type='demand')
        model.objective = 'DM_{}'.format(Product)
#         sol = md.optimize()
        return model

def set_SubiUma(model, Sub, Flux, Ratio):
    Catoms = [countCinFormula(model.metabolites.get_by_id(subi).formula) for subi in Sub]
    Cup = Flux/np.sum(Catoms)
#         print(sub,Cup)
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
    if len(Sub)<2:
#         print(Sub, Cup)
        model.reactions.get_by_id('EX_{}'.format(Sub[0])).lower_bound = -Cup
    else:
#         print('else ', len(sub))
        C1 = Catoms[0]
        C2 = Catoms[1]
        F1,F2 = CoSub(Flux, C1, C2, Ratio)
#         print(F1, F2)
        for subi, fluxi in zip(Sub, [F1,F2]):
            model.reactions.get_by_id('EX_{}'.format(subi)).lower_bound = -fluxi
#         print(md.slim_optimize())
    return model

In [3]:
# generating cobra variable from SBML/xml file
# assuming the base path is in 'Simulation'
iUma_File = 'iUma22.xml' # os.path.join('..','iUL_versions','iUL959_112718.xml')
model = read_sbml_model(iUma_File)

## Adding missing reactions

In [4]:
# Mannitol production
# https://biocyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWY-3881
ID = 'PseudoMan6P2Mnl'
if ID not in model.reactions:
    reaction = Reaction(ID)
    reaction.name = 'Pseudoreaction from Mannose-6-P to Mannitol'
    reaction.lower_bound = 0.
    reaction.upper_bound = 1000.
#     reaction.annotation = {'biocyc':'MANNPISOM-RXN', 'brenda': '5.3.1.8'}
    model.add_reactions([reaction])
    model.reactions.get_by_id(ID).build_reaction_from_string('MANNOSE-6P_c + nadph_c + h_c + h2o_c --> mnl_c + nadp_c + pi_c')

    print(model.reactions.get_by_id(ID).id)
    print(model.reactions.get_by_id(ID).reaction)
    print(model.reactions.get_by_id(ID).check_mass_balance())
    
model.add_boundary(model.metabolites.get_by_id('2obut_e'), type="exchange", lb=0)
model.add_boundary(model.metabolites.get_by_id('for_e'), type="exchange", lb=0)

PseudoMan6P2Mnl
MANNOSE-6P_c + h2o_c + h_c + nadph_c --> mnl_c + nadp_c + pi_c
{}


Reaction identifier,EX_for_e
Name,formate exchange
Memory address,0x07fd9a31b0910
Stoichiometry,for_e --> formate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [5]:
# Searching for metabolite IDs
CheckMet = 'riacyl'
[print('{}:\t{}:\t{}:\t{}\n'.format(MetID.name, MetID.id, MetID.formula, MetID.annotation)) for MetID in model.metabolites.query(CheckMet, 'name')];

In [6]:
ProList = ['itacon_c','succ_c','mal__L_c', 'eryth_c', 'mnl_c','hex2manerythol_c','UA_A_c', 'CPD-15924_c'] # hexACP_c -> UA_A_c, CPD-13014_c: tributyrin
SubList = [['xyl__D_e'], ['glc__D_e'], ['2obut_e'], ['glc__D_e', 'for_e'], ['glc__D_e', 'ac_e']] #['ac_e'], ['for_e'], 
SubProCom = [(x,y) for x in SubList for y in ProList]

In [7]:
Solution_df = pd.DataFrame(columns=['Substrate', 'Product', 'Yield'])
for SubProi in SubProCom:
    Substrate_lst = SubProi[0]
    Product_str = SubProi[1]
    CnumPro = countCinFormula(model.metabolites.get_by_id(Product_str).formula)
    F = 100 # C-mole/gDW/h
    R = 5
    with model as md:
        md = set_SubiUma(md, Substrate_lst, F, R)
        md = set_ObjiUma(md, Product_str)
        sol = md.optimize()
        Yield = sol.fluxes['DM_{}'.format(Product_str)]*CnumPro/F
        Solution_df = Solution_df.append({'Substrate': str(Substrate_lst), 'Product': Product_str, 'Yield': round(Yield,2)}, ignore_index=True)
        mybnd = [md.reactions.get_by_id('EX_{}'.format(subi)).lower_bound for subi in Substrate_lst]
        print('{} production on {}: {:.2f}'.format(Product_str, Substrate_lst, Yield))

itacon_c production on ['xyl__D_e']: 1.00
succ_c production on ['xyl__D_e']: 1.00
mal__L_c production on ['xyl__D_e']: 1.00
eryth_c production on ['xyl__D_e']: 0.79
mnl_c production on ['xyl__D_e']: 0.89
hex2manerythol_c production on ['xyl__D_e']: 0.86
UA_A_c production on ['xyl__D_e']: 0.83
CPD-15924_c production on ['xyl__D_e']: 0.73
itacon_c production on ['glc__D_e']: 1.00
succ_c production on ['glc__D_e']: 1.00
mal__L_c production on ['glc__D_e']: 1.00
eryth_c production on ['glc__D_e']: 0.88
mnl_c production on ['glc__D_e']: 0.92
hex2manerythol_c production on ['glc__D_e']: 0.86
UA_A_c production on ['glc__D_e']: 0.83
CPD-15924_c production on ['glc__D_e']: 0.73
itacon_c production on ['2obut_e']: 0.83
succ_c production on ['2obut_e']: 1.00
mal__L_c production on ['2obut_e']: 1.00
eryth_c production on ['2obut_e']: 0.32
mnl_c production on ['2obut_e']: 0.37
hex2manerythol_c production on ['2obut_e']: 0.39
UA_A_c production on ['2obut_e']: 0.46
CPD-15924_c production on ['2obut_e

In [8]:
Solution_Tab = pd.pivot(Solution_df, values='Yield', index='Substrate', columns='Product')
Solution_Tab['Mean-Substrates'] = round(Solution_Tab.mean(axis=1), 2)
Mean = Solution_Tab.mean()
Mean.name='Mean-Products'
Solution_Tab = round(Solution_Tab.append(Mean), 2)
Solution_Tab

Product,CPD-15924_c,UA_A_c,eryth_c,hex2manerythol_c,itacon_c,mal__L_c,mnl_c,succ_c,Mean-Substrates
Substrate,,,,,,,,,
['2obut_e'],0.46,0.46,0.32,0.39,0.83,1.0,0.37,1.0,0.60
"['glc__D_e', 'ac_e']",0.70,0.80,0.73,0.83,1.00,1.0,0.82,1.0,0.86
"['glc__D_e', 'for_e']",0.61,0.69,0.73,0.72,1.00,1.0,0.77,1.0,0.82
['glc__D_e'],0.73,0.83,0.88,0.86,1.00,1.0,0.92,1.0,0.90
['xyl__D_e'],0.73,0.83,0.79,0.86,1.00,1.0,0.89,1.0,0.89
Mean-Products,0.65,0.72,0.69,0.73,0.97,1.0,0.75,1.0,0.81


In [9]:
Solution_Tab.to_excel('{}_SubProdTable.xlsx'.format(Today))

In [10]:
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -100/6
    model.reactions.get_by_id('EX_for_e').lower_bound = -0
    model = set_ObjiUma(model, 'succ_c')
#     model.objective = 'EX_succ_e'
    print(model.summary())

Objective
1.0 DM_succ_c = 25.000000000000178

Uptake
------
Metabolite    Reaction  Flux  C-Number  C-Flux
  glc__D_e EX_glc__D_e 16.67         6 100.00%
      o2_e     EX_o2_e  12.5         0   0.00%

Secretion
---------
Metabolite  Reaction Flux  C-Number  C-Flux
    succ_c DM_succ_c  -25         4 100.00%
     h2o_e  EX_h2o_e  -25         0   0.00%

